In [1]:
import sys
sys.path.append("..")

In [2]:
import cupy as cp

x = cp.arange(6).reshape(2, 3)

x

array([[0, 1, 2],
       [3, 4, 5]])

In [3]:
y = x.sum(axis=1)
y

array([ 3, 12])

In [4]:
import numpy as np
import cupy as cp

n = np.array([1, 2, 3])
c = cp.asarray(n)

assert type(c) == cp.ndarray

In [5]:
c = cp.array([1, 2, 3])
n = cp.asnumpy(c)
assert type(n) == np.ndarray

In [6]:
x = np.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == np

x = cp.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == cp

In [9]:
import time

import mytorch
import mytorch.functions as F
from mytorch import optimizers, DataLoader
from mytorch.models import MLP

max_epoch = 5
batch_size = 100

hidden_size = 1000
train_set = mytorch.datasets.MNIST(train=True)
test_set = mytorch.datasets.MNIST(train=False)

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, hidden_size, 10))
optim = optimizers.SGD(model.parameters)

print("not using GPU")
    
train_loss_saver = []
test_loss_saver = []
train_acc_saver = []
test_acc_saver = []


for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    for x, t in train_loader:
        pred = model(x)
        loss = F.softmax_cross_entropy(pred, t)
        acc = F.accuracy(pred, t)
        
        model.zerograd()
        loss.backward()
        optim.step()
        
        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)
    train_loss_saver.append((epoch, sum_loss / len(train_set)))
    train_acc_saver.append((epoch, sum_acc / len(train_set)))
    print(f"[{epoch+1}/{max_epoch}] train_loss: {sum_loss / len(train_set)}, "
          f"accuracy: {sum_acc / len(train_set)}")
    
    sum_loss, sum_acc = 0, 0
    with mytorch.no_grad():
        for x, t in test_loader:
            pred = model(x)
            loss = mytorch.functions.softmax_cross_entropy(pred, t)
            acc = mytorch.functions.accuracy(pred, t)
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)
        test_loss_saver.append((epoch, sum_loss / len(test_set)))
        test_acc_saver.append((epoch, sum_acc / len(test_set)))
        print(f"test_loss: {sum_loss / len(test_set)}, "
              f"accuracy: {sum_acc / len(test_set)}")
    
    print(f"elapse times : {time.time() - start}")


not using GPU
[1/5] train_loss: 2.2670746636390686, accuracy: 0.1744
test_loss: 2.2247932720184327, accuracy: 0.1748
elapse times : 7.385754823684692
[2/5] train_loss: 2.1723104294141136, accuracy: 0.36628333333333335
test_loss: 2.104503791332245, accuracy: 0.517
elapse times : 6.0160486698150635
[3/5] train_loss: 2.0241631187995277, accuracy: 0.5096
test_loss: 1.9135946869850158, accuracy: 0.4951
elapse times : 6.045778512954712
[4/5] train_loss: 1.78109676361084, accuracy: 0.6079166666666667
test_loss: 1.6147808802127839, accuracy: 0.6482
elapse times : 6.117432355880737
[5/5] train_loss: 1.4714031920830408, accuracy: 0.6770833333333334
test_loss: 1.3059992212057114, accuracy: 0.7352
elapse times : 6.076480150222778


In [10]:
import time

import mytorch
import mytorch.functions as F
from mytorch import optimizers, DataLoader
from mytorch.models import MLP

max_epoch = 5
batch_size = 100

hidden_size = 1000
train_set = mytorch.datasets.MNIST(train=True)
test_set = mytorch.datasets.MNIST(train=False)

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, hidden_size, 10))
optim = optimizers.SGD(model.parameters)

if mytorch.cuda.gpu_enable:
    print("using GPU")
    train_loader.to_gpu()
    test_loader.to_gpu()
    model.to_gpu()

    
train_loss_saver = []
test_loss_saver = []
train_acc_saver = []
test_acc_saver = []


for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    for x, t in train_loader:
        pred = model(x)
        loss = F.softmax_cross_entropy(pred, t)
        acc = F.accuracy(pred, t)
        
        model.zerograd()
        loss.backward()
        optim.step()
        
        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)
    train_loss_saver.append((epoch, sum_loss / len(train_set)))
    train_acc_saver.append((epoch, sum_acc / len(train_set)))
    print(f"[{epoch+1}/{max_epoch}] train_loss: {sum_loss / len(train_set)}, "
          f"accuracy: {sum_acc / len(train_set)}")
    
    sum_loss, sum_acc = 0, 0
    with mytorch.no_grad():
        for x, t in test_loader:
            pred = model(x)
            loss = mytorch.functions.softmax_cross_entropy(pred, t)
            acc = mytorch.functions.accuracy(pred, t)
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)
        test_loss_saver.append((epoch, sum_loss / len(test_set)))
        test_acc_saver.append((epoch, sum_acc / len(test_set)))
        print(f"test_loss: {sum_loss / len(test_set)}, "
              f"accuracy: {sum_acc / len(test_set)}")
    print(f"elapse times : {time.time() - start}")


using GPU
[1/5] train_loss: 2.2655182472719306, accuracy: 0.17836666666666667
test_loss: 2.224961286230049, accuracy: 0.1358
elapse times : 5.637044906616211
[2/5] train_loss: 2.1678672456258536, accuracy: 0.36741666666666667
test_loss: 2.097939332704492, accuracy: 0.4365
elapse times : 5.357033014297485
[3/5] train_loss: 2.013332341243734, accuracy: 0.5107833333333334
test_loss: 1.8958331486127955, accuracy: 0.643
elapse times : 5.3505449295043945
[4/5] train_loss: 1.7633395323662284, accuracy: 0.6083333333333333
test_loss: 1.5931521402023174, accuracy: 0.6747
elapse times : 5.3990960121154785
[5/5] train_loss: 1.4502346608947838, accuracy: 0.6776833333333333
test_loss: 1.2931785370676505, accuracy: 0.6486
elapse times : 5.393986940383911
